Данные из файла Volgmed_2013.xls (конкретные данные выбрать в соответствии с вариантом) очистить от выбросов и пропусков, найти коэффициенты линейной регрессии с их пи-значениями и коэффициент детерминации. Исключить регрессоры, у которых пи-значения больше 0,05, вычислить заново коэффициенты линейной регрессии с их пи-значениями и коэффициент детерминации, а также выборочную дисперсию регрессионных остатков. Провести кросс-валидацию, разбив 200 раз случайным образом в борку на обучающую (80%) и контрольную (20%) группы. Посчитать среднее значение квадрата отклонения по контрольной группе и его отношение к выборочной дисперсии регрессионных остатков.

Вариант 4. Сила правой руки юношей первого курса в зависимости от экскурсии грудной клетки, окружности грудной клетки в покое, челночного бега.

In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def data_cleanup(data : pd.DataFrame):
    def is_value_fit(value):
        return (lower_limit < value < upper_limit)
    cleaned_data = data.copy()
    cleaned_data = cleaned_data.dropna()
    data_array = cleaned_data.to_numpy(dtype=float)
    for col in range(len(data_array[0])):
        q_025 = np.quantile(data_array[:,col],0.25)
        q_075 = np.quantile(data_array[:,col],0.75)
        iqd = q_075 - q_025
        upper_limit = q_075 + 1.5 * iqd
        lower_limit = q_025 - 1.5 * iqd
        cleaned_data = cleaned_data[cleaned_data.iloc[:,col].apply(is_value_fit)]
        print()
    return cleaned_data

In [7]:
volgmed = pd.read_excel("Volgmed_2013.xlsx", header = 1)
data = volgmed[((volgmed.Пол == 'муж.') | (volgmed.Пол == 'муж')) & (volgmed.Курс == 1)][['Сила правой руки, кг',
                                                                                          'Экскурсия грудной клетки, см',
                                                                                          'Окружность грудной клетки в покое, см',
                                                                                          'Челночный бег, с']]

data = data[(data['Челночный бег, с'] != 'осв') & (data['Челночный бег, с'] != 'осв.')]

data

,"Сила правой руки, кг","Экскурсия грудной клетки, см","Окружность грудной клетки в покое, см","Челночный бег, с"
389,52.0,6.0,84,19.9
392,34.0,5.0,85,20.5
395,80.0,8.0,119,21.2
398,51.0,11.0,80,15
401,28.0,6.0,77,20.5
...,...,...,...,...
4087,40.0,10.0,87,21.5
4089,58.0,9.0,86,21.3
4091,33.0,6.0,86,21.3
4093,40.0,7.0,90,21.7


In [8]:
clean_data = data_cleanup(data)

clean_data

,"Сила правой руки, кг","Экскурсия грудной клетки, см","Окружность грудной клетки в покое, см","Челночный бег, с"
389,52.0,6.0,84,19.9
392,34.0,5.0,85,20.5
401,28.0,6.0,77,20.5
404,68.0,8.0,82,20.7
407,60.0,5.0,90,19.2
...,...,...,...,...
4087,40.0,10.0,87,21.5
4089,58.0,9.0,86,21.3
4091,33.0,6.0,86,21.3
4093,40.0,7.0,90,21.7


In [9]:
strength = clean_data['Сила правой руки, кг'].to_numpy(dtype = float)
X1 = sm.add_constant(clean_data['Экскурсия грудной клетки, см'].to_numpy(dtype = float))
X2 = np.column_stack((X1, clean_data['Окружность грудной клетки в покое, см'].to_numpy(dtype = float)))
X3 = np.column_stack((X2, clean_data['Челночный бег, с'].to_numpy(dtype = float)))
model = sm.OLS(strength, X3)
res = model.fit()
print(res.summary())
print('Коэффициент детерминации', res.rsquared)
print('Коэффициенты с их оценкой пи-значений:')
names = ['Константа', 'Экскурсия грудной клетки, см', 'Окружность грудной клетки в покое, см', 'Челночный бег, с']
params_to_delete = []
for i in range(len(res.params)):
    print(f"{names[i]}: {res.params[i]} (p-значение: {res.pvalues[i]})")
    if(res.pvalues[i] > 0.05):
        params_to_delete.append(i)
params_to_delete

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     12.79
Date:                Wed, 06 Nov 2024   Prob (F-statistic):           8.46e-08
Time:                        15:13:21   Log-Likelihood:                -964.73
No. Observations:                 251   AIC:                             1937.
Df Residuals:                     247   BIC:                             1952.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         62.4429     19.214      3.250      0.0

[]

In [10]:
for p in params_to_delete:
    X3 = np.delete(X3, p, axis=1)
    names = np.delete(names, p)
model = sm.OLS(strength, X3)
res = model.fit()
print(res.summary())
print('Коэффициент детерминации', res.rsquared)
print('Коэффициенты с их оценкой пи-значений:')
params_to_delete = []
for i in range(len(res.params)):
    print(f"{names[i]}: {res.params[i]} (p-значение: {res.pvalues[i]})")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     12.79
Date:                Wed, 06 Nov 2024   Prob (F-statistic):           8.46e-08
Time:                        15:13:21   Log-Likelihood:                -964.73
No. Observations:                 251   AIC:                             1937.
Df Residuals:                     247   BIC:                             1952.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         62.4429     19.214      3.250      0.0

In [11]:
residuals = res.resid
sample_variance = np.var(residuals, ddof = 1)
print(f"Выборочная дисперсия остатков: {sample_variance}")

Выборочная дисперсия остатков: 128.1519442204582


In [12]:
mse_list = []
for i in range(200):
    X_train, X_test, y_train, y_test = train_test_split(X3, strength, test_size=0.2)
    X_train_df = pd.DataFrame(X_train, columns = ['Константа', 'Экскурсия грудной клетки, см', 'Окружность грудной клетки в покое, см', 'Челночный бег, с'])
    X_test_df = pd.DataFrame(X_test, columns = ['Константа', 'Экскурсия грудной клетки, см', 'Окружность грудной клетки в покое, см', 'Челночный бег, с'])
    model = sm.OLS(y_train, X_train_df)
    model = model.fit()
    y_pred = model.predict(X_test_df)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)
mean_mse = np.mean(mse_list)
residuals = model.resid
sample_variance = np.var(residuals, ddof = 1)
print(f"Среднее значение квадрата отклонения: {mean_mse}")
print(f"Выборочная дисперсия остатков: {sample_variance}")
print(f"Отношение MSE к выборочной дисперсии: {mean_mse / sample_variance}")

Среднее значение квадрата отклонения: 132.3270259947405
Выборочная дисперсия остатков: 129.04638881661498
Отношение MSE к выборочной дисперсии: 1.0254221540657567
